# Low-Level Spectrum Viewer — Intensity + Spectrum Direct Control

This notebook demonstrates building spectra using the low-level `Intensity` and `Spectrum` classes directly, without `Molecule` or `MoleculeDict` wrappers. You work with raw line lists, calculate intensities manually, and construct wavelength grids from scratch.

## What This Notebook Covers:
1. **Single Molecule Explorer** — Interactive spectrum generation with full parameter control
2. **Multi-Molecule Comparison** — Overlay multiple species with normalization and offset options
3. **Temperature Sequence** — Visualize how spectra evolve across temperature ranges

This approach gives you complete control over the spectral synthesis pipeline and is useful for:
- Understanding the low-level calculation workflow
- Custom spectral modeling scenarios
- Educational demonstrations of radiative transfer physics

In [1]:
# First, add the iSLAT package to the Python path
import sys
from pathlib import Path

# Navigate from notebook location to the iSLATTests directory
notebook_dir = Path.cwd()
islat_root = notebook_dir.parent.parent.parent # Interactive Widgets -> Notebooks -> Examples -> iSLATTests
if str(islat_root) not in sys.path:
    sys.path.insert(0, str(islat_root))

# Core libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba

# Use the interactive widget backend
%matplotlib widget

# IPython widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# iSLAT low-level data types
from iSLAT.Modules.DataTypes import Intensity, Spectrum
from iSLAT.Modules.DataTypes.MoleculeLineList import MoleculeLineList

print("Imports successful!")
print(f"matplotlib backend: {matplotlib.get_backend()}")

Imports successful!
matplotlib backend: widget


In [2]:
# --- Available molecules and their properties ---

data_dir = Path(islat_root) / "iSLAT" / "DATAFILES" / "HITRANdata"

# Molecule configurations with suggested default temperatures
MOLECULE_CONFIG = {
    "H2O": {
        "file": "data_Hitran_H2O.par",
        "color": "blue",
        "label": "$H_2O$",
        "default_temp": 850,
    },
    "CO": {
        "file": "data_Hitran_CO.par",
        "color": "red",
        "label": "CO",
        "default_temp": 1000,
    },
    "CO2": {
        "file": "data_Hitran_CO2.par",
        "color": "green",
        "label": "$CO_2$",
        "default_temp": 300,
    },
    "HCN": {
        "file": "data_Hitran_HCN.par",
        "color": "purple",
        "label": "HCN",
        "default_temp": 700,
    },
    "C2H2": {
        "file": "data_Hitran_C2H2.par",
        "color": "orange",
        "label": "$C_2H_2$",
        "default_temp": 600,
    },
    "CH4": {
        "file": "data_Hitran_CH4.par",
        "color": "brown",
        "label": "$CH_4$",
        "default_temp": 500,
    },
    "NH3": {
        "file": "data_Hitran_NH3.par",
        "color": "teal",
        "label": "$NH_3$",
        "default_temp": 400,
    },
}

# Pre-load line lists for all molecules
line_lists = {}
intensity_calcs = {}

print("Loading molecular line lists...")
for mol_name, config in MOLECULE_CONFIG.items():
    file_path = data_dir / config["file"]
    if file_path.exists():
        try:
            ll = MoleculeLineList(molecule_id=mol_name, filename=str(file_path))
            line_lists[mol_name] = ll
            intensity_calcs[mol_name] = Intensity(ll)
            print(f"   {mol_name}: {len(ll.get_wavelengths())} lines")
        except Exception as e:
            print(f"   {mol_name}: Failed to load - {e}")
    else:
        print(f"   {mol_name}: File not found")

print(f"\nLoaded {len(line_lists)} molecules: {list(line_lists.keys())}")

Loading molecular line lists...
   H2O: 305561 lines
   CO: 1334 lines
   CO2: 173129 lines
   HCN: 127972 lines
   C2H2: 81454 lines
   CH4: 322978 lines
   NH3: 91416 lines

Loaded 7 molecules: ['H2O', 'CO', 'CO2', 'HCN', 'C2H2', 'CH4', 'NH3']


## Single Molecule Explorer

Select a molecule and explore its spectrum with interactive parameter controls.

In [3]:
# --- Single Molecule Spectrum Explorer ---

single_output = widgets.Output()

# Molecule selector
mol_dropdown = widgets.Dropdown(
    options=list(line_lists.keys()),
    value="H2O",
    description='Molecule:',
    style={'description_width': '100px'},
)

# Parameter sliders
single_temp_slider = widgets.IntSlider(
    value=850, min=100, max=3000, step=50,
    description='Temperature (K):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

single_log_n_slider = widgets.FloatSlider(
    value=18.0, min=14.0, max=22.0, step=0.25,
    description='log_10 N (cm^-2):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

single_radius_slider = widgets.FloatSlider(
    value=0.5, min=0.05, max=3.0, step=0.05,
    description='Radius (AU):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

single_wave_slider = widgets.FloatRangeSlider(
    value=[10.0, 20.0],
    min=2.0, max=30.0,
    step=0.5,
    description='λ range (μm):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

single_info = widgets.HTML(value="")

def on_molecule_change(change):
    """Update temperature slider to molecule default when selection changes."""
    mol_name = mol_dropdown.value
    if mol_name in MOLECULE_CONFIG:
        single_temp_slider.value = MOLECULE_CONFIG[mol_name]["default_temp"]
    update_single_spectrum(None)

def update_single_spectrum(change):
    """Generate and display spectrum for selected molecule."""
    mol_name = mol_dropdown.value
    if mol_name not in line_lists:
        single_info.value = f"<b style='color:red'>Error: {mol_name} not loaded</b>"
        return
    
    config = MOLECULE_CONFIG[mol_name]
    intensity_calc = intensity_calcs[mol_name]
    
    temp = float(single_temp_slider.value)
    n_mol = 10 ** single_log_n_slider.value
    radius = single_radius_slider.value
    lam_min, lam_max = single_wave_slider.value
    
    # Fixed parameters
    distance = 160
    R = 3000
    dv = 1.0
    area = np.pi * radius**2
    
    # Calculate
    intensity_calc.calc_intensity(t_kin=temp, n_mol=n_mol, dv=dv)
    
    spectrum = Spectrum(
        lam_min=lam_min, lam_max=lam_max, dlambda=0.001,
        R=R, distance=distance
    )
    spectrum.add_intensity(intensity_calc, area)
    
    wavelengths = spectrum.lamgrid
    flux = spectrum.flux_jy
    peak = np.max(flux)
    
    single_info.value = (
        f"<b>{config['label']}</b> | T={temp:.0f} K | N=10^{single_log_n_slider.value:.1f} cm^-2 | "
        f"R={radius:.2f} AU | Peak flux: {peak:.4f} Jy"
    )
    
    with single_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 5))
        
        color = config['color']
        ax.plot(wavelengths, flux, color=color, linewidth=0.5)
        ax.fill_between(wavelengths, 0, flux, alpha=0.3, color=color)
        
        ax.set_xlabel('Wavelength (μm)')
        ax.set_ylabel('Flux (Jy)')
        ax.set_title(f'{config["label"]} Spectrum (T={temp:.0f}K, N={n_mol:.0e} cm^-2)')
        ax.set_xlim(lam_min, lam_max)
        ax.set_ylim(0, peak * 1.15 if peak > 0 else 0.01)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Connect
mol_dropdown.observe(on_molecule_change, names='value')
for slider in [single_temp_slider, single_log_n_slider, single_radius_slider, single_wave_slider]:
    slider.observe(update_single_spectrum, names='value')

# Initial display
update_single_spectrum(None)

display(widgets.VBox([
    single_info,
    mol_dropdown,
    single_temp_slider,
    single_log_n_slider,
    single_radius_slider,
    single_wave_slider,
]))
display(single_output)

Output()

## Multi-Molecule Comparison

Compare spectra from multiple molecules simultaneously with individual parameter controls.

In [4]:
# --- Multi-Molecule Comparison ---

compare_output = widgets.Output()

# Molecule checkboxes
mol_checkboxes = {}
for mol_name in line_lists.keys():
    config = MOLECULE_CONFIG.get(mol_name, {})
    mol_checkboxes[mol_name] = widgets.Checkbox(
        value=(mol_name in ["H2O", "CO"]),  # Default selection
        description=config.get('label', mol_name),
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100px'),
    )

# Common parameters
compare_temp_slider = widgets.IntSlider(
    value=850, min=100, max=2000, step=50,
    description='Temperature (K):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

compare_log_n_slider = widgets.FloatSlider(
    value=18.0, min=15.0, max=20.0, step=0.25,
    description='log_10 N (cm^-2):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

compare_wave_slider = widgets.FloatRangeSlider(
    value=[10.0, 18.0],
    min=2.0, max=30.0,
    step=0.5,
    description='λ range (μm):',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px'),
)

normalize_checkbox = widgets.Checkbox(
    value=False,
    description='Normalize to peak',
    style={'description_width': 'initial'},
)

offset_checkbox = widgets.Checkbox(
    value=False,
    description='Offset spectra vertically',
    style={'description_width': 'initial'},
)

compare_info = widgets.HTML(value="")

def update_comparison(change=None):
    """Generate and compare spectra for selected molecules."""
    selected = [name for name, cb in mol_checkboxes.items() if cb.value]
    
    if not selected:
        compare_info.value = "<b>Select at least one molecule</b>"
        return
    
    temp = float(compare_temp_slider.value)
    n_mol = 10 ** compare_log_n_slider.value
    lam_min, lam_max = compare_wave_slider.value
    
    # Fixed parameters
    distance = 160
    R = 3000
    dv = 1.0
    radius = 0.5
    area = np.pi * radius**2
    
    # Calculate spectra for each molecule
    spectra = {}
    max_flux = 0
    
    for mol_name in selected:
        if mol_name not in intensity_calcs:
            continue
            
        intensity_calc = intensity_calcs[mol_name]
        intensity_calc.calc_intensity(t_kin=temp, n_mol=n_mol, dv=dv)
        
        spectrum = Spectrum(
            lam_min=lam_min, lam_max=lam_max, dlambda=0.001,
            R=R, distance=distance
        )
        spectrum.add_intensity(intensity_calc, area)
        
        spectra[mol_name] = {
            'wavelengths': spectrum.lamgrid,
            'flux': spectrum.flux_jy,
            'peak': np.max(spectrum.flux_jy),
        }
        max_flux = max(max_flux, spectra[mol_name]['peak'])
    
    # Info
    labels = [MOLECULE_CONFIG.get(m, {}).get('label', m) for m in selected]
    compare_info.value = (
        f"<b>Comparing:</b> {', '.join(labels)}<br>"
        f"T={temp:.0f} K | N=10^{compare_log_n_slider.value:.1f} cm^-2 | R={radius} AU"
    )
    
    # Plot
    with compare_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12, 6))
        
        offset_step = max_flux * 0.3 if offset_checkbox.value else 0
        
        for i, mol_name in enumerate(selected):
            if mol_name not in spectra:
                continue
                
            data = spectra[mol_name]
            config = MOLECULE_CONFIG.get(mol_name, {'color': 'gray', 'label': mol_name})
            
            flux = data['flux'].copy()
            
            # Normalize if requested
            if normalize_checkbox.value and data['peak'] > 0:
                flux = flux / data['peak']
            
            # Apply vertical offset
            offset = i * offset_step
            flux_plot = flux + offset
            
            ax.plot(data['wavelengths'], flux_plot, 
                    color=config['color'], linewidth=0.7, 
                    label=f"{config['label']} (peak: {data['peak']:.3f} Jy)")
            ax.fill_between(data['wavelengths'], offset, flux_plot, 
                           alpha=0.2, color=config['color'])
        
        ax.set_xlabel('Wavelength (μm)')
        ylabel = 'Normalized Flux' if normalize_checkbox.value else 'Flux (Jy)'
        if offset_checkbox.value:
            ylabel += ' + offset'
        ax.set_ylabel(ylabel)
        ax.set_title(f'Multi-Molecule Comparison (T={temp}K, N={n_mol:.0e} cm^-2)')
        ax.set_xlim(lam_min, lam_max)
        ax.legend(loc='upper right', fontsize=9)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Connect all widgets
for cb in mol_checkboxes.values():
    cb.observe(update_comparison, names='value')
for slider in [compare_temp_slider, compare_log_n_slider, compare_wave_slider]:
    slider.observe(update_comparison, names='value')
normalize_checkbox.observe(update_comparison, names='value')
offset_checkbox.observe(update_comparison, names='value')

# Initial
update_comparison()

# Layout
checkbox_row = widgets.HBox(list(mol_checkboxes.values()))
options_row = widgets.HBox([normalize_checkbox, offset_checkbox])

display(widgets.VBox([
    compare_info,
    widgets.HTML("<b>Select molecules to compare:</b>"),
    checkbox_row,
    compare_temp_slider,
    compare_log_n_slider,
    compare_wave_slider,
    options_row,
]))
display(compare_output)

Output()

## Temperature Sequence Animation

See how a molecule's spectrum evolves across a range of temperatures.

In [5]:
# --- Temperature Sequence for a Single Molecule ---

temp_seq_output = widgets.Output()

# Molecule selector
seq_mol_dropdown = widgets.Dropdown(
    options=list(line_lists.keys()),
    value="H2O",
    description='Molecule:',
    style={'description_width': '100px'},
)

# Temperature range
seq_temp_slider = widgets.IntRangeSlider(
    value=[300, 1500],
    min=100, max=3000,
    step=100,
    description='T range (K):',
    continuous_update=False,
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px'),
)

# Number of temperature steps
n_temps_slider = widgets.IntSlider(
    value=5, min=2, max=10, step=1,
    description='# of temps:',
    continuous_update=False,
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px'),
)

seq_wave_slider = widgets.FloatRangeSlider(
    value=[12.0, 18.0],
    min=2.0, max=30.0,
    step=0.5,
    description='λ range (μm):',
    continuous_update=False,
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px'),
)

seq_info = widgets.HTML(value="")

def update_temp_sequence(change=None):
    """Generate spectra across a temperature range."""
    mol_name = seq_mol_dropdown.value
    if mol_name not in intensity_calcs:
        seq_info.value = f"<b style='color:red'>Error: {mol_name} not loaded</b>"
        return
    
    config = MOLECULE_CONFIG.get(mol_name, {'color': 'blue', 'label': mol_name})
    intensity_calc = intensity_calcs[mol_name]
    
    temp_min, temp_max = seq_temp_slider.value
    n_temps = n_temps_slider.value
    lam_min, lam_max = seq_wave_slider.value
    
    temperatures = np.linspace(temp_min, temp_max, n_temps).astype(int)
    
    # Fixed parameters
    n_mol = 1e18
    distance = 160
    R = 3000
    dv = 1.0
    radius = 0.5
    area = np.pi * radius**2
    
    # Calculate spectra
    spectra = []
    max_flux = 0
    
    for temp in temperatures:
        intensity_calc.calc_intensity(t_kin=float(temp), n_mol=n_mol, dv=dv)
        spectrum = Spectrum(
            lam_min=lam_min, lam_max=lam_max, dlambda=0.001,
            R=R, distance=distance
        )
        spectrum.add_intensity(intensity_calc, area)
        spectra.append({
            'temp': temp,
            'wavelengths': spectrum.lamgrid,
            'flux': spectrum.flux_jy,
        })
        max_flux = max(max_flux, np.max(spectrum.flux_jy))
    
    seq_info.value = (
        f"<b>{config['label']} Temperature Sequence</b><br>"
        f"Temperatures: {', '.join(str(t) for t in temperatures)} K<br>"
        f"N=10^18 cm^-2 | R={radius} AU | d={distance} pc"
    )
    
    # Create colormap from cool (blue) to hot (red)
    cmap = plt.cm.coolwarm
    norm = plt.Normalize(temp_min, temp_max)
    
    with temp_seq_output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(20, 6))
        
        for spec_data in spectra:
            color = cmap(norm(spec_data['temp']))
            ax.plot(spec_data['wavelengths'], spec_data['flux'], 
                    color=color, linewidth=0.8, 
                    label=f"T = {spec_data['temp']} K")
        
        ax.set_xlabel('Wavelength (μm)')
        ax.set_ylabel('Flux (Jy)')
        ax.set_title(f'{config["label"]} Spectrum vs Temperature')
        ax.set_xlim(lam_min, lam_max)
        ax.set_ylim(0, max_flux * 1.15 if max_flux > 0 else 0.01)
        ax.legend(loc='upper right', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # Add colorbar
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, label='Temperature (K)')
        
        plt.tight_layout()
        plt.show()

# Connect
seq_mol_dropdown.observe(update_temp_sequence, names='value')
seq_temp_slider.observe(update_temp_sequence, names='value')
n_temps_slider.observe(update_temp_sequence, names='value')
seq_wave_slider.observe(update_temp_sequence, names='value')

# Initial
update_temp_sequence()

display(widgets.VBox([
    seq_info,
    seq_mol_dropdown,
    seq_temp_slider,
    n_temps_slider,
    seq_wave_slider,
]))
display(temp_seq_output)

Output()